In [13]:
import pandas as pd
import plotly.express as pe
import plotly.graph_objects as go
import plotly.io as pio
import matplotlib.pyplot as plt
import numpy as np
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
import rasterio
from rasterio.plot import show
from rasterio.mask import mask
from shapely.geometry import box
from feature_engineer import smart_imputation, create_advanced_features

In [9]:
#Generate the data and visualize it
# Load the reference band for spatial information
with rasterio.open('LC08_L2SP_116050_20240306_20240315_02_T1_SR_B5.TIF') as src:
    nir_data = src.read(1)
    transform = src.transform
    crs = src.crs
    height, width = src.shape

# Create a DataFrame with spatial information
def create_spatial_dataframe(sampled_rows, sampled_cols, transform, crs):
    """Create DataFrame with spatial coordinates and predictions"""

    data = []
    for i, (row, col) in enumerate(zip(sampled_rows, sampled_cols)):
        # Convert pixel coordinates to geographic coordinates
        lon, lat = transform * (col, row)

        data.append({
            'pixel_id': i,
            'row': row,
            'col': col,
            'longitude': lon,
            'latitude': lat,
            'x_coord': col,
            'y_coord': row
        })

    return pd.DataFrame(data)

# Create spatial DataFrame
spatial_df = create_spatial_dataframe(sampled_rows, sampled_cols, transform, crs)
print(f"Spatial DataFrame shape: {spatial_df.shape}")
print(spatial_df.head())

Spatial DataFrame shape: (10000, 7)
   pixel_id   row   col  longitude   latitude  x_coord  y_coord
0         0  5592  2728   234225.0  1548255.0     2728     5592
1         1  5012  1998   212325.0  1565655.0     1998     5012
2         2  2842  2313   221775.0  1630755.0     2313     2842
3         3  1025  1784   205905.0  1685265.0     1784     1025
4         4  4703  5453   315975.0  1574925.0     5453     4703


In [10]:
# Dictionary to store band data
# Load all bands and add to DataFrame
band_files = {
    'B1_coastal': 'LC08_L2SP_116050_20240306_20240315_02_T1_SR_B1.TIF',
    'B2_blue': 'LC08_L2SP_116050_20240306_20240315_02_T1_SR_B2.TIF',
    'B3_green': 'LC08_L2SP_116050_20240306_20240315_02_T1_SR_B3.TIF',
    'B4_red': 'LC08_L2SP_116050_20240306_20240315_02_T1_SR_B4.TIF',
    'B5_nir': 'LC08_L2SP_116050_20240306_20240315_02_T1_SR_B5.TIF',
    'B6_swir1': 'LC08_L2SP_116050_20240306_20240315_02_T1_SR_B6.TIF',
    'B7_swir2': 'LC08_L2SP_116050_20240306_20240315_02_T1_SR_B7.TIF'
}

# Add band values to DataFrame
for band_name, file_path in band_files.items():
    print(f"Loading {band_name}...")
    with rasterio.open(file_path) as src:
        band_data = src.read(1)

    band_values = []
    for row, col in zip(sampled_rows, sampled_cols):
        band_values.append(band_data[row, col])

    spatial_df[band_name] = band_values

print(f"DataFrame with bands: {spatial_df.shape}")
print(spatial_df[['B1_coastal', 'B2_blue', 'B3_green', 'B4_red', 'B5_nir', 'B6_swir1', 'B7_swir2']].head())

Loading B1_coastal...
Loading B2_blue...
Loading B3_green...
Loading B4_red...
Loading B5_nir...
Loading B6_swir1...
Loading B7_swir2...
DataFrame with bands: (10000, 14)
   B1_coastal  B2_blue  B3_green  B4_red  B5_nir  B6_swir1  B7_swir2
0        7850     7910      7848    7281    7070      7375      7454
1        8579     8451      8070    7526    7217      7404      7452
2        9379     9623     10447    9809   20860     13671     10281
3        8513     8869     10169   10612   17190     15152     12005
4        7643     8623     10432    9509    7261      7367      7443


In [11]:
def calculate_spectral_indices(df):
    """Calculate spectral indices and add to DataFrame"""

    df = df.copy()

    # NDVI - Vegetation index
    df['NDVI'] = (df['B5_nir'] - df['B4_red']) / (df['B5_nir'] + df['B4_red'])

    # NDBI - Built-up index
    df['NDBI'] = (df['B6_swir1'] - df['B5_nir']) / (df['B6_swir1'] + df['B5_nir'])

    # NDWI - Water index
    df['NDWI'] = (df['B3_green'] - df['B5_nir']) / (df['B3_green'] + df['B5_nir'])

    # SAVI - Soil Adjusted Vegetation Index
    L = 0.5  # soil brightness correction factor
    df['SAVI'] = ((df['B5_nir'] - df['B4_red']) / (df['B5_nir'] + df['B4_red'] + L)) * (1 + L)

    # Brightness
    df['brightness'] = (df['B4_red'] + df['B5_nir'] + df['B6_swir1']) / 3

    # Band ratios
    df['ratio_swir_nir'] = df['B6_swir1'] / df['B5_nir']
    df['ratio_nir_red'] = df['B5_nir'] / df['B4_red']
    df['ratio_swir_red'] = df['B6_swir1'] / df['B4_red']

    # Advanced urban indices
    df['UI'] = (df['B6_swir1'] - df['B5_nir']) / (df['B6_swir1'] + df['B5_nir'])  # Urban Index
    df['IBI'] = (2 * df['B6_swir1'] / (df['B6_swir1'] + df['B5_nir']) -
                (df['B5_nir'] / (df['B5_nir'] + df['B4_red']) +
                 df['B4_red'] / (df['B4_red'] + df['B3_green']))) / \
               (2 * df['B6_swir1'] / (df['B6_swir1'] + df['B5_nir']) +
                (df['B5_nir'] / (df['B5_nir'] + df['B4_red']) +
                 df['B4_red'] / (df['B4_red'] + df['B3_green'])))

    return df

# Add spectral indices
spatial_df = calculate_spectral_indices(spatial_df)
print(f"DataFrame with indices: {spatial_df.shape}")
print(spatial_df[['NDVI', 'NDBI', 'NDWI', 'SAVI', 'brightness']].head())

DataFrame with indices: (10000, 24)
       NDVI      NDBI      NDWI      SAVI    brightness
0  4.551948  0.021115  0.052152  6.827684   7242.000000
1  4.424269  0.012790  0.055799  6.636179   7382.333333
2  0.360331  1.689699  1.760724  0.540488  14780.000000
3  0.236602  1.963329  2.138784  0.354896  14318.000000
4  3.773882  0.007246  0.179223  5.660654   8045.666667


In [12]:
with rasterio.open('LC08_L2SP_116050_20240306_20240315_02_T1_QA_PIXEL.TIF') as src:
    qa_data = src.read(1)

    qa_values = []
    for row, col in zip(sampled_rows, sampled_cols):
        qa_values.append(qa_data[row, col])

    spatial_df['QA_pixel'] = qa_values

    # Decode QA band (simplified)
    spatial_df['cloud_mask'] = (spatial_df['QA_pixel'] & (1 << 5)) != 0
    spatial_df['cloud_shadow_mask'] = (spatial_df['QA_pixel'] & (1 << 3)) != 0
    spatial_df['water_mask'] = (spatial_df['QA_pixel'] & (1 << 7)) != 0


In [22]:
df = create_advanced_features(smart_imputation(spatial_df))

#Finally, add the dummy binary target
df["is_urban"] = 0

#Export the data
df = df.drop(columns=["pixel_id", "row", "col", "longitude", "latitude", "x_coord", "y_coord"], axis=1)
df.to_csv("manila_testing_landsat_data.csv", index=False)

In [23]:
df

,B1_coastal,B2_blue,B3_green,B4_red,B5_nir,B6_swir1,B7_swir2,NDVI,NDBI,NDWI,SAVI,brightness,ratio_swir_nir,ratio_nir_red,ratio_swir_red,UI,IBI,QA_pixel,cloud_mask,cloud_shadow_mask,water_mask,EBBI,ratio_swir2_nir,ratio_red_blue,ratio_green_blue,ratio_swir1_blue,sum_swir_bands,diff_nir_red,B5_nir_neighbor_mean,B5_nir_neighbor_std,B6_swir1_neighbor_mean,B6_swir1_neighbor_std,NDBI_neighbor_mean,NDBI_neighbor_std,NDVI_neighbor_mean,NDVI_neighbor_std,NDBI_x_NDVI,NDBI_div_NDVI,B6_swir1_x_B5_nir,B6_swir1_div_B5_nir,brightness_x_ratio_swir_nir,brightness_div_ratio_swir_nir,NDBI,NDVI,B6_swir1,B5_nir,NDBI NDVI,NDBI B6_swir1,NDBI B5_nir,NDVI B6_swir1,NDVI B5_nir,B6_swir1 B5_nir,log_B6_swir1,log_B5_nir,log_B4_red,log_brightness,is_urban
0,7850.0,7910.0,7848.0,7281.0,7070.0,7375.0,7454.0,4.551948,0.021115,0.052152,6.827684,7242.000000,1.043140,0.971020,1.012910,0.021115,0.023661,21952.0,0.0,0.0,1.0,0.014038,1.054314,0.920480,0.992162,0.932364,14829.0,-211.0,13132.4682,5625.216858,11216.2251,3904.609022,1.138891,8.129291,1.366601,2.120602,0.096112,0.004639,52141250.0,1.043140,7554.420085,6942.500339,0.021115,4.551948,7375.0,7070.0,0.096112,155.719972,149.280028,33570.613546,32182.269528,52141250.0,8.905987,8.863757,8.893161,8.887791,0
1,8579.0,8451.0,8070.0,7526.0,7217.0,7404.0,7452.0,4.424269,0.012790,0.055799,6.636179,7382.333333,1.025911,0.958942,0.983790,0.012790,0.020510,21952.0,0.0,0.0,1.0,0.008444,1.032562,0.890545,0.954917,0.876109,14856.0,-309.0,13132.4682,5625.216858,11216.2251,3904.609022,1.138891,8.129291,1.366601,2.120602,0.056586,0.002891,53434668.0,1.025911,7573.617293,7195.880560,0.012790,4.424269,7404.0,7217.0,0.056586,94.695848,92.304152,32757.288747,31929.950417,53434668.0,8.909911,8.884333,8.926252,8.906980,0
2,9379.0,9623.0,10447.0,9809.0,20860.0,13671.0,10281.0,0.360331,1.689699,1.760724,0.540488,14780.000000,0.655369,2.126618,1.393720,-0.208190,-0.190472,22080.0,0.0,0.0,0.0,-0.162134,0.492857,1.019329,1.085628,1.420659,23952.0,11051.0,13132.4682,5625.216858,11216.2251,3904.609022,1.138891,8.129291,1.366601,2.120602,0.608851,4.689293,285177060.0,0.655369,9686.355705,22552.176139,1.689699,0.360331,13671.0,20860.0,0.608851,23099.876546,35247.123454,4926.088917,7516.510483,285177060.0,9.523105,9.945637,9.191158,9.601098,0
3,8513.0,8869.0,10169.0,10612.0,17190.0,15152.0,12005.0,0.236602,1.963329,2.138784,0.354896,14318.000000,0.881443,1.619864,1.427818,-0.063014,-0.092923,21824.0,0.0,0.0,0.0,-0.047446,0.698371,1.196527,1.146578,1.708423,27157.0,6578.0,13132.4682,5625.216858,11216.2251,3904.609022,1.138891,8.129291,1.366601,2.120602,0.464527,8.298037,260462880.0,0.881443,12620.496568,16243.823918,1.963329,0.236602,15152.0,17190.0,0.464527,29748.367324,33749.632676,3584.988706,4067.182936,260462880.0,9.625954,9.752141,9.269835,9.569343,0
4,7643.0,8623.0,10432.0,9509.0,7261.0,7367.0,7443.0,3.773882,0.007246,0.179223,5.660654,8045.666667,1.014599,0.763592,0.774740,0.007246,0.050814,21952.0,0.0,0.0,1.0,0.004392,1.025065,1.102748,1.209788,0.854343,14810.0,-2248.0,13132.4682,5625.216858,11216.2251,3904.609022,1.138891,8.129291,1.366601,2.120602,0.027347,0.001920,53491787.0,1.014599,8163.121655,7929.901679,0.007246,3.773882,7367.0,7261.0,0.027347,53.384058,52.615942,27802.188193,27402.156708,53491787.0,8.904902,8.890411,9.160099,8.993013,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,8092.0,8397.0,9516.0,9681.0,17599.0,15962.0,11838.0,0.290249,1.903966,2.118864,0.435366,14414.000000,0.906983,1.817891,1.648797,-0.048777,-0.094351,21824.0,0.0,0.0,0.0,-0.037857,0.672652,1.152912,1.133262,1.900917,27800.0,7918.0,13132.4682,5625.216858,11216.2251,3904.609022,1.138891,8.129291,1.366601,2.120602,0.552625,6.559761,280915238.0,0.906983,13073.258026,15892.243203,1.903966,0.290249,15962.0,17599.0,0.552625,30391.103900,33507.896100,4632.958798,5108.09684